# Tutorial 1: Salmonella dataset

## Background

This tutorial consists of an index constructed from a *Salmonella enterica* dataset previously published in (https://doi.org/10.1128/jcm.02200-15). To prepare the data, the genomes were downloaded, downsampled (to min 30x coverage) to reduce data size and run through [snippy](https://github.com/tseemann/snippy) to identify SNVs. These files were then loaded into a genomics index using the `gdi load snippy` command.

## Getting data

Let's first download the data for this tutorial. To do this please run the below commands in your shell:

```bash
wget ...
unzip salmonella-project.zip
```

Great. Now that we've got some data, let's explore the command-line interface to access this data. Note that in a Jupyter Python notebook, prepending a command with `!` runs the command in a shell instead of the Python interpreter (e.g., `!ls` runs the command `ls`).

## Command-line interface (`gdi`)

### List samples

The below command lists the samples loaded in this index. Note that instead of passing the project with `--project-dir` you can change to the project directory and `gdi` will figure out which project you're in.

In [1]:
!gdi --project-dir salmonella-project list samples | head -n 5

SH09-29
SH10-015
SH12-001
SH13-001
SH14-003


### List reference genomes

This lists all the loaded reference genomes in this genomics index. This can be useful for commands later which require you to pass the name of the reference genome.

In [2]:
!gdi --project-dir salmonella-project list genomes

S_HeidelbergSL476


### Query for a particular mutation

This searches for a particular mutation (in this case a deletion of a `CG` at position 2865334 and an insertion of a `G`). If you are familiar with the VCF format, this is given in terms of `CHROM:POS:REF:ALT` format.

In [3]:
!gdi --project-dir salmonella-project query mutation 'gi|194447306|ref|NC_011083.1|:2865334:CG:C' | column -s$'\t' -t

Type      Feature                                     Sample Name  Sample ID  Status
mutation  gi|194447306|ref|NC_011083.1|:2865334:CG:C  SH12-013     9          Present


## Build an alignment

To build an alignment for use in further phylogenetics software you can do the following.

In [4]:
!gdi --project-dir salmonella-project build alignment --reference-name S_HeidelbergSL476 --align-type full --output-file out.aln

Wrote alignment to [out.aln]


In [5]:
!head -n 2 out.aln

>SH08-001 generated automatically
AGAGANTACGTNTGGTTGCAAGAGATCATAACAGGGGGAATTAGTTGAAAATAAATATAT


This will produce an alignment with length equal to the reference genome, masking any missing data or gaps with `N`, and concatenating individual sequences (contigs) in the alignment together to construct a single, whole-genome alignment.

You can pass in specific samples to include with `--sample`.

# Python API (Genomics Data Index API)

Now let's move on to the Python interface for loading, and querying data in this index. This is a much more powerful (and flexible) way to work with your data which attempts to provide seamless data flow between this index and Python [pandas](https://pandas.pydata.org/).

We first start out by connecting to our project through `GenomicsDataIndex`.

In [6]:
import genomics_data_index.api as gdi

db = gdi.GenomicsDataIndex.connect('salmonella-project')
db

<GenomicsDataIndex(samples=59)>

Great. We're connected. The `samples=59` tells us how many samples are in this database.

You can run `db.sample_names()` or `db.reference_names()` to list the samples and reference genomes in this database.

In [7]:
db.sample_names()[0:5]

['SH09-29', 'SH10-015', 'SH12-001', 'SH13-001', 'SH14-003']

In [8]:
db.reference_names()

['S_HeidelbergSL476']

## List all features (mutations)

To list a summary of all indexed mutations we can do:

In [9]:
db.mutations_summary(reference_genome='S_HeidelbergSL476')

,Sequence,Position,Deletion,Insertion,Count
Mutation,,,,,
gi|194447306|ref|NC_011083.1|:34780:A:G,gi|194447306|ref|NC_011083.1|,34780,A,G,3
gi|194447306|ref|NC_011083.1|:43651:G:T,gi|194447306|ref|NC_011083.1|,43651,G,T,1
gi|194447306|ref|NC_011083.1|:58804:T:A,gi|194447306|ref|NC_011083.1|,58804,T,A,10
gi|194447306|ref|NC_011083.1|:63393:C:A,gi|194447306|ref|NC_011083.1|,63393,C,A,13
gi|194447306|ref|NC_011083.1|:70519:T:TG,gi|194447306|ref|NC_011083.1|,70519,T,TG,6
...,...,...,...,...,...
gi|194447306|ref|NC_011083.1|:4860641:G:A,gi|194447306|ref|NC_011083.1|,4860641,G,A,57
gi|194447306|ref|NC_011083.1|:4876176:A:G,gi|194447306|ref|NC_011083.1|,4876176,A,G,58
gi|194447306|ref|NC_011083.1|:4882099:C:T,gi|194447306|ref|NC_011083.1|,4882099,C,T,57


This gives us back a DataFrame summarizing the mutations.


## Search for a particular mutation

We can search for genomic samples containing particular mutation by starting a query and using the `has()` method.

In [10]:
q = db.samples_query().has('gi|194447306|ref|NC_011083.1|:63393:C:A', kind='mutation')
q

<SamplesQueryIndex[22% (13/59) samples]>

The `has()` method can be read as "select samples that **have** this particular mutation". The selected samples is printed above (13/59 have this particular mutation).

The specific samples can be shown with the `toframe()` method:

In [11]:
q.toframe()

,Query,Sample Name,Sample ID,Status
0,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-001,3,Present
1,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-003,13,Present
2,gi|194447306|ref|NC_011083.1|:63393:C:A,SH11-001,15,Present
3,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-002,18,Present
4,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-004,24,Present
5,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-008,37,Present
6,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-011,38,Present
7,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-005,40,Present
8,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-010,41,Present
9,gi|194447306|ref|NC_011083.1|:63393:C:A,SH12-006,45,Present


Or you can use `summary()`

In [12]:
q.summary()

,Query,Present,Absent,Unknown,Total,% Present,% Absent,% Unknown
0,gi|194447306|ref|NC_011083.1|:63393:C:A,13,46,<NA>,59,22.033898,77.966102,<NA>


Or you can use `tolist()`

In [13]:
q.tolist()

['SH12-001',
 'SH12-003',
 'SH11-001',
 'SH12-002',
 'SH12-004',
 'SH12-008',
 'SH12-011',
 'SH12-005',
 'SH12-010',
 'SH12-006',
 'SH12-009',
 'SH12-007',
 'SH10-001']

## Chaining queries

Queries can be chained together to select samples that match every criteria given in the `has()` method.

In [14]:
q = db.samples_query() \
    .has('gi|194447306|ref|NC_011083.1|:63393:C:A', kind='mutation') \
    .has('gi|194447306|ref|NC_011083.1|:4876176:A:G', kind='mutation')
q

<SamplesQueryIndex[22% (13/59) samples]>

This can be read as "select all samples that **have** 63393 C to A **AND** select all samples thave **have** 4876176 A to G".

## Searching within a tree

Queries are not limited to `has()`. We can also use `within()` to select samples that match criteria (often selecting samples within some certain region of a phylogenetic tree).

## Attach external metadata

So far we've been looking at only the genomics data. But often times many details insights can be derived from the associated metadata with the genomic samples. External metadata can be attached and tracked by our queries. This also gives us annother method for selecting samples using pandas selection statements (e.g., `metadata['Column'] == 'value'`).

To attach external metadata we first must load it up in Python as a DataFrame (note `head(3)` just means only print the first 3 rows, which avoids printing a very large table for this tutorial).

In [15]:
import pandas as pd

metadata_df = pd.read_csv('salmonella-project/metadata.tsv', sep='\t', dtype=str)
metadata_df.head(3)

,Strain,Source,Isolation date,Outbreak number,PFGE pattern,PT,Canadian PFGE pattern
0,SH12-001,Human,05-2012,1,2,19,SHXAI.0001/SHBNI.0001
1,SH12-002,Human,05-2012,1,2,19,SHXAI.0001/SHBNI.0001
2,SH12-003,Human,05-2012,1,2,19,SHXAI.0001/SHBNI.0001


Now we can attach to our query using the `join()` method:

In [16]:
# Setup a new query (you don't have to do this, but this makes sure the results are all as expected in the tutorial)
q = db.samples_query().has('gi|194447306|ref|NC_011083.1|:4860641:G:A', kind='mutation')

# Join our query with the given data frame
q = q.join(metadata_df, sample_names_column='Strain')
q

<DataFrameSamplesQuery[97% (57/59) samples]>

To join we had to define a column containing the sample names. We now get back a query of type `DataFrameSamplesQuery`.

## Query external metadata

We can continue using this to select samples and when we're done use the `toframe()` method to dump out our selected data as a DataFrame.

In [17]:
q.has('gi|194447306|ref|NC_011083.1|:70519:T:TG', kind='mutation').toframe()

,Query,Sample Name,Sample ID,Status,Strain,Source,Isolation date,Outbreak number,PFGE pattern,PT,Canadian PFGE pattern
0,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH13-001,4,Present,SH13-001,Human,11-2013,2,2,26,SHXAI.0001/SHBNI.0001
1,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH13-003,32,Present,SH13-003,Human,11-2013,2,2,26,SHXAI.0001/SHBNI.0001
2,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH13-004,25,Present,SH13-004,Human,11-2013,2,2,26,SHXAI.0001/SHBNI.0001
3,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH13-005,47,Present,SH13-005,Human,11-2013,2,2,26,SHXAI.0001/SHBNI.0001
4,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH13-006,54,Present,SH13-006,Human,11-2013,2,2,26,SHXAI.0001/SHBNI.0001
5,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH13-007,16,Present,SH13-007,Human,11-2013,2,2,26,SHXAI.0001/SHBNI.0001


Running `toframe()` will add some additional columns to the front of the external data frame defining the genomics query information.

## Selecting by data frame expressions

You can also select samples within the query using expressions involving data frame columns. For example, if we wanted to select only samples where **Source** is `Food` we could do:

In [18]:
q.has(metadata_df['Source'] == 'Food', kind='dataframe').toframe().head(3)

,Query,Sample Name,Sample ID,Status,Strain,Source,Isolation date,Outbreak number,PFGE pattern,PT,Canadian PFGE pattern
0,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH12-009,50,Present,SH12-009,Food,05-2012,1,2,19,SHXAI.0001/SHBNI.0001
1,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH12-010,41,Present,SH12-010,Food,05-2012,1,2,19,SHXAI.0001/SHBNI.0001
2,gi|194447306|ref|NC_011083.1|:4860641:G:A AND ...,SH14-013,42,Present,SH14-013,Food,08-2014,3,2,19,SHXAI.0001/SHBNI.0001


# Putting it all together

So far we've seen connecting to an index (project), querying by mutations and by relationships in a tree, as well as attaching external metadata. We can put this all together and do some basic visualiations of our selections on the tree (using the `ete3` toolkit) like so: